# LAB 6 - Matheus Gomes Maia
## ATIVIDADE


* 1-Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

* 2-Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; (10 pts)

* 3-Implementar as seguintes versões do modelo vetorial:(10 pts)
  * Representação binária;
  * TF (lembre-se que esse modelo já está implementado);
  * TF-IDF;
  * BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho).

* 4-Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta; (10 pts)

* 5-Compare os resultados encontrados e responda. (20 pts)

    * 5.1-Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.
    * 5.2 Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard (Links to an external site.)Links to an external site.)

* Teste alguns valores diferentes de k e reporte como os resultados foram afetados.



## SETUP

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import math
import re
from scipy.optimize import minimize_scalar
import seaborn as sns
from nltk.corpus import stopwords
import operator

pd.set_option("display.max_rows",40)

%matplotlib inline
nltk.download('punkt')

nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/maia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/maia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1-Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [11]:
data = pd.read_csv('../results_fornecida.csv')
data = data.drop_duplicates(subset='url', keep='last')


#Pre process news
# Tokenize, Join and Filter
# Words into a new data Frame
txt = [str(news) for news in data["text"].tolist()]
words = [nltk.word_tokenize(sentence) for sentence in txt]
words = [item for sublist in words for item in sublist] 
words = [word.lower() for word in words if (word.isalpha() and len(word)) > 2] 

# New Data frame with Word, Frequency and Ranking columns 
words_df = pd.DataFrame(words, columns=['word']) #All words
word_counts = words_df.word.value_counts().reset_index() #Join by word. Adds Frequency
word_counts.columns = ['Word', 'Freq'] #Naming columns 
word_counts['word_rank'] = word_counts.Freq.rank(ascending=False) #Adds ranking

def parse(news):
    words = [nltk.word_tokenize(news)]
    words = [item for sublist in words for item in sublist] 
    words = [word.lower() for word in words if (word.isalpha() and len(word)) > 2]
    return words
    

def buidIndexTF(txt):
    I = {}
    n = 0
    for news in txt:
        T = parse(news)
        for token in set(T):
            if(token not in I.keys()):
                I[token] = [(n, T.count(token))]
            else:
                I[token].append((n, T.count(token)))
        n += 1
    return I

invIndexTF = buidIndexTF(txt)

## 2-Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; (10 pts)

IDF(W) = log[(M+1)/k]

In [12]:
def parse(news):
    words = [nltk.word_tokenize(news)]
    words = [item for sublist in words for item in sublist] 
    words = [word.lower() for word in words if (word.isalpha() and len(word)) > 2]
    return words
    
def idf(M, k):
    return math.log((M+1)/k)
    
M = len(txt)
def buidIndexTFIDF(txt):
    I = {}
    n = 0
    for news in txt:
        T = parse(news)
        for token in set(T):
            k = T.count(token)
            if(token not in I.keys()):
                I[token] = [(n, k, idf(M, k))]
            else:
                I[token].append((n, k, idf(M, k)))
        n += 1
    return I

indexTFIDF = buidIndexTFIDF(txt)

In [13]:
indexTFIDF["juíza"]

[(0, 2, 4.8283137373023015), (1, 1, 5.521460917862246)]

##  3-Implementar as seguintes versões do modelo vetorial:(10 pts)
  * Representação binária;
  * TF (lembre-se que esse modelo já está implementado);
  * TF-IDF;
  * BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho).


In [14]:
#REPRESENTAÇÃO BINÁRIA
def buidBinaryIndex(txt):
    I = {}
    n = 0
    for news in txt:
        T = parse(news)
        for token in set(T):
            k = T.count(token)
            if(token not in I.keys()):
                I[token] = [(n)]
            else:
                I[token].append((n))
        n += 1
    return I

binaryIndex = buidBinaryIndex(txt)
binaryIndex["juíza"]

[0, 1]

In [15]:
def termAtTimeBinary(Q, Inv, k):
    L = { key: Inv[key] for key in Q if key in Inv.keys() } #Filtra entrys de interesse no IndexInvertido
    R = {}
    for term in set(L.keys()): # Para cada termo
            for doc in L[term]: #  Itera pelos docs que tem o termo
                R[doc] = 1
                
                
    #Return K
    if(k > len(R.keys())):
        k = len(R.keys())
    return (sorted(R, key=lambda k: R[k], reverse=True))[:k]

def termAtTimeTF(Q, Inv, k):
    L = { key: Inv[key] for key in Q if key in Inv.keys() } #Filtra entrys de interesse no IndexInvertido
    R = {}
    for term in set(L.keys()): # Para cada termo
            for doc in L[term]: #  Itera pelos docs que tem o termo
                # Acumula o score
                if(doc[0] not in R.keys()):
                    R[doc[0]] = doc[1]*Q.count(term)
                else:
                    R[doc[0]] += doc[1]*Q.count(term)
                
    #Return K
    if(k > len(R.keys())):
        k = len(R.keys())
    return (sorted(R, key=lambda k: R[k], reverse=True))[:k]



def termAtTimeTFIDF(Q, Inv, k):
    L = { key: Inv[key] for key in Q if key in Inv.keys() } #Filtra entrys de interesse no IndexInvertido
    R = {}
    for term in set(L.keys()): # Para cada termo
            for doc in L[term]: #  Itera pelos docs que tem o termo
                # Acumula o score
                if(doc[0] not in R.keys()):
                    R[doc[0]] = Q.count(term)*doc[1]*doc[2]
                else:
                    R[doc[0]] += Q.count(term)*doc[1]*doc[2]
                
                
    #Return K
    if(k > len(R.keys())):
        k = len(R.keys())
    return (sorted(R, key=lambda k: R[k], reverse=True))[:k]


Kconst = 3
def BM25(Kconst, cwq, cwd, idf):
    #print(Kconst, cwq, cwd, idf)
    BM25Result = cwd*(((Kconst+1)*cwd)/(cwd + Kconst))*idf
    #print(BM25Result)
    return BM25Result

def termAtTimeBM25(Q, Inv, k):
    L = { key: Inv[key] for key in Q if key in Inv.keys() } #Filtra entrys de interesse no IndexInvertido
    R = {}
    for term in L.keys(): # Para cada termo
            for doc in L[term]: #  Itera pelos docs que tem o termo
                # Acumula o score
                if(doc[0] not in R.keys()):
                    R[doc[0]] = BM25(Kconst, Q.count(term), doc[1], doc[2])
                else:
                    R[doc[0]] += BM25(Kconst, Q.count(term), doc[1], doc[2])
                
                
    #Return K
    if(k > len(R.keys())):
        k = len(R.keys())
    return (sorted(R, key=lambda k: R[k], reverse=True))[:k]


## 4-Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta; (10 pts)


In [16]:
querys = ["bolsonaro armas", "inglaterra país", "quanto com dele israel"]

#BINARY
for query in querys:
    print(query)
    print("BINARY")
    print(termAtTimeBinary(query.split(), binaryIndex, 5))
    
    print("TF")
    print(termAtTimeTF(query.split(), invIndexTF, 5))
    
    print("TF-IDF")
    print(termAtTimeTFIDF(query.split(), indexTFIDF, 5))
    
    print("BM25")
    print(termAtTimeBM25(query.split(), indexTFIDF, 5))
    
    print()

bolsonaro armas
BINARY
[0, 1, 213, 22, 6]
TF
[150, 206, 165, 18, 21]
TF-IDF
[150, 206, 165, 21, 18]
BM25
[150, 206, 165, 18, 21]

inglaterra país
BINARY
[0, 2, 4, 5, 7]
TF
[150, 165, 172, 207, 106]
TF-IDF
[150, 165, 172, 106, 207]
BM25
[150, 165, 172, 207, 106]

quanto com dele israel
BINARY
[0, 1, 2, 3, 4]
TF
[248, 68, 62, 165, 150]
TF-IDF
[68, 248, 150, 165, 62]
BM25
[248, 68, 150, 165, 62]



## 5-Compare os resultados encontrados e responda. (20 pts)

    * 5.1-Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.
    * 5.2 Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)

In [17]:
# Sobre a consulta: "quanto com dele israel"
print(txt[248][:500], "\n")
print(txt[68][:500])

“A única coisa que desejamos é sair daqui. Não podem nos condenar por cuidar da casa e de nossos filhos no Estado Islâmico”  dizem Yolanda Martínez  Luna Fernández e Lubna Miludi. São cidadãs espanholas que viajaram com seus maridos para a Síria em 2014 e sobreviveram ao desmoronamento do califado do  (EI) em seu último reduto de Baguz  oásis na fronteira oriental da Síria com o Iraque. Elas conversam com o EL PAÍS num casebre do campo de acolhida de Hol  onde 73.000 pessoas estão retidas em con 

 (Norfolk  1953)  ex-assessor estratégico do presidente dos Estados Unidos    e um dos pais do novo populismo político  foi perguntado na quinta-feira passada  21  na Biblioteca Angelica em Roma  se era o diabo. “Deixo as pessoas decidirem por si mesmas”  respondeu  devolvendo o poder ao povo  como dita seu cânone político. Muitos na capital italiana  onde se tornou um habitual suspeito dos salões conservadores  afirmam sentir o cheiro de enxofre quando ele sai da sala. Acontece nos círculos 

Acredito que o modelo BM25 trouxe melhores resultados, embora só exista uma diferença entre os retornos de BM25 e de TF-IDF nas consultas realizadas. 

Na consulta "quanto com dele israel" o primeiro retorno utilizando BM25 fala sobre assuntos do oriente médio, enquanto que na consulta com TF-IDF esse retorno está em segundo lugar, o retorno que está em primeiro lugat no modelo TF-IDF não é tão relevante.

In [18]:
from statistics import mean 



querys = ["bolsonaro armas", "inglaterra país", "quanto com dele israel"]
#Utilizando TOP 10
top = 10


def mBIN(query):
    return termAtTimeBinary(query, binaryIndex, top)
def mTF(query):
    return termAtTimeTF(query, invIndexTF, top)
def mTFIDF(query):
    return termAtTimeTFIDF(query, indexTFIDF, top)
def mB25(query):
    return termAtTimeBM25(query, indexTFIDF, top)



models = [mBIN, mTF, mTFIDF, mB25]

mapModel = {}

#BINARY
for query in querys:
    for m1 in models:
        for m2 in models:
            if m1 != m2:
                r1 = m1(query.split())
                r2 = m2(query.split())
                #print(r1, r2)
                intersection = set(r1).intersection(set(r2))
                union = set(r1).union(set(r2))
                mkey = m1.__name__+" "+m2.__name__
                makey = m2.__name__+" "+m1.__name__
                value = len(intersection)/len(union)
                
                if(makey in mapModel.keys()):
                    continue
                if(mkey in mapModel.keys()):
                    mapModel[mkey].append(round(value,2))
                else:
                    mapModel[mkey] = [round(value,2)]

for key in sorted(mapModel.keys()):
    print("FOR PAIR MODEL",key, " "*(12-len(key)),  "Jaccard Index MEAN", "%.2f" % mean(mapModel[key]), "LIST:", mapModel[key])
                

FOR PAIR MODEL mBIN mB25     Jaccard Index MEAN 0.03 LIST: [0.05, 0.05, 0.0]
FOR PAIR MODEL mBIN mTF      Jaccard Index MEAN 0.03 LIST: [0.05, 0.05, 0.0]
FOR PAIR MODEL mBIN mTFIDF   Jaccard Index MEAN 0.02 LIST: [0.0, 0.05, 0.0]
FOR PAIR MODEL mTF mB25      Jaccard Index MEAN 0.94 LIST: [1.0, 1.0, 0.82]
FOR PAIR MODEL mTF mTFIDF    Jaccard Index MEAN 0.72 LIST: [0.67, 0.82, 0.67]
FOR PAIR MODEL mTFIDF mB25   Jaccard Index MEAN 0.77 LIST: [0.67, 0.82, 0.82]


Podemos perceber que o modelos TF e BM25 estão próximos e que o modelo Binário não é proximo a nenhum outro.

Se utilizarmos um top menor que 5 os modelos TF, TFIDF e BM25 ficam bem próximos. 